In [5]:
search_for = 121
start_from = 0
threads = 10
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004935264587402344
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 15
all cases: 237758115
type: [1, 1, 1, 1, 121] 121
cases of this type: 214358881
100000 13.937456391055395
200000 17.85469031359432
300000 16.768446253466514
400000 18.116673461430043
500000 18.099580826517993
600000 20.291750752328763
700000 18.155043164753547
800000 17.96653125641152
900000 10.446172245743076
1000000 15.186642496028622
1100000 17.984791650002013
1200000 17.02961625697899
1300000 17.620237612020034
1400000 18.24913112504322
1500000 16.772349629142862
1600000 18.731225000197092
1700000 17.970081258737583
1800000 9.303046751980624
1900000 16.77252478461378
2000000 21.473164354401572
2100000 20.987630857807783
2200000 20.72688517888337
2300000 20.56328298035197
2400000 20.223611676967273
2500000 20.314248565774243
2600000 21.185347539427507
2700000 8.200470572296322
2800000 20.392119540033256
2900000 20.22160134855988
3000000 

29400000 14.193145069119097
29500000 14.228921587348692
29600000 16.48091532081467
29700000 15.598931157778493
29800000 16.355669207748896
29900000 10.13969745553312
30000000 15.78340340307355
30100000 16.007139778803786
30200000 15.45845797599778
30300000 16.557819819232048
30400000 8.92357722019961
30500000 15.769293780821043
30600000 15.894745032143364
30700000 15.292232394134897
30800000 14.86423487034737
30900000 13.893365803343654
31000000 10.179611640108433
31100000 10.24422359371465
31200000 13.072170072903754
31300000 14.797482155212995
31400000 15.348430836780718
31500000 14.760240380910247
31600000 16.02615671900945
31700000 8.875510860476528
31800000 15.327072511574775
31900000 15.661672247214156
32000000 15.649574215912358
32100000 15.58813198102671
32200000 7.677037767536845
32300000 16.752454448453314
32400000 15.192457055565198
32500000 16.039542369605588
32600000 15.071474119275809
32700000 14.470469886572243
32800000 7.362642939388061
32900000 13.949171731393186
33000

59000000 6.340510130528744
59100000 12.363085444694827
59200000 12.121043747875385
59300000 13.03631263706056
59400000 5.3254417128076454
59500000 12.498324776325891
59600000 12.593538534523702
59700000 12.512536997474852
59800000 7.128917398817963
59900000 12.934261071982782
60000000 13.252708408890506
60100000 13.471254096753396
60200000 13.204825698180029
60300000 14.129911643590937
60400000 13.598354120041737
60500000 14.024702595590005
60600000 12.49408717642167
60700000 13.199597605398498
60800000 5.678713824927432
60900000 11.753945722428153
61000000 12.912130245858156
61100000 10.854385554882562
61200000 6.936749036754747
61300000 12.413782801295383
61400000 11.808584958517354
61500000 9.352717920502007
61600000 9.875554170858482
61700000 12.74886948726706
61800000 13.395854222797444
61900000 13.216168335308046
62000000 12.821542773822788
62100000 13.44271585961476
62200000 14.366511227344105
62300000 13.250923752945281
62400000 13.037289732242167
62500000 11.291379115442606
62

88700000 12.276235839036563
88800000 10.960153052512448
88900000 10.153801572973867
89000000 10.266440955514597
89100000 10.661981541720866
89200000 9.990974609769312
89300000 9.273631636137896
89400000 6.579471875590235
89500000 4.319690572469989
89600000 9.319003768405146
89700000 7.215914051940283
89800000 9.997974046962097
89900000 10.365623264123993
90000000 10.978840859864981
90100000 10.575438396370192
90200000 10.3678907368706
90300000 11.272167341389835
90400000 11.899673965637193
90500000 11.17328605630986
90600000 10.786571713908135
90700000 10.254256449096737
90800000 10.072819126937578
90900000 10.385881425449659
91000000 10.106340408642335
91100000 7.481485399285009
91200000 7.683436000343621
91300000 4.498250711436957
91400000 5.785479644122027
91500000 10.408898230971385
91600000 10.212110436696653
91700000 9.98904050514809
91800000 10.521709674802471
91900000 10.599991066792679
92000000 10.628393485980965
92100000 11.45249530290386
92200000 12.005583601345693
92300000 

117900000 2.7795366349046136
118000000 6.240750800221413
118100000 7.860831102295015
118200000 8.219097896038742
118300000 8.590829471743335
118400000 8.061044087600711
118500000 8.022365459495177
118600000 8.300389279762577
118700000 8.841527301133649
118800000 8.78691684441185
118900000 8.374392844831997
119000000 8.147986556245415
119100000 7.773940347413063
119200000 7.968545632141689
119300000 8.022534978841792
119400000 7.637493380527198
119500000 4.923209683765335
119600000 3.819574092291726
119700000 4.009901520788372
119800000 6.564948181306908
119900000 8.46494566808221
120000000 7.825648830506175
120100000 7.832749892550826
120200000 7.90893040567911
120300000 8.200241827876171
120400000 8.12985443838048
120500000 8.950775859765029
120600000 9.096890016658048
120700000 7.926678066961159
120800000 8.15310918539318
120900000 8.48005861822173
121000000 7.976286937617968
121100000 8.50830182198935
121200000 8.034407071518531
121300000 4.69994075064324
121400000 4.147134681348642

147200000 6.634508391670773
147300000 6.875187843962829
147400000 6.941254024459263
147500000 6.175691638440807
147600000 4.380454329367697
147700000 5.7350824159516165
147800000 6.367565712002311
147900000 5.941487538294553
148000000 3.537712718267951
148100000 6.179913223457591
148200000 6.550231062428037
148300000 3.172436484733846
148400000 6.5955188483505145
148500000 6.416678976497004
148600000 7.012024965891113
148700000 7.740039781978551
148800000 6.969853839402527
148900000 6.8832243144720655
149000000 6.90117175357613
149100000 6.555101852622499
149200000 6.454063798465348
149300000 6.521120654858898
149400000 3.4816485497228102
149500000 6.017259314183702
149600000 5.943618697771897
149700000 4.259942229017705
149800000 4.719445891143127
149900000 6.546809435290112
150000000 6.208613072840929
150100000 3.0330715239553916
150200000 6.0451463070037725
150300000 6.496951926876574
150400000 6.780731012120518
150500000 7.315874423584082
150600000 6.719564948373229
150700000 6.633

176400000 4.092026363097718
176500000 4.124482439849151
176600000 4.210049687064314
176700000 4.432286011153907
176800000 4.518807988548832
176900000 2.342468548708482
177000000 4.798955106686006
177100000 4.931703020923584
177200000 4.519343647314794
177300000 4.567245737654845
177400000 4.6071572330043615
177500000 2.55682845201335
177600000 4.487516721105516
177700000 4.423183347417467
177800000 4.110659207490471
177900000 4.109155174282144
178000000 4.271406529907088
178100000 1.873698732985672
178200000 4.0736367092072365
178300000 4.148024801559217
178400000 4.484472362631576
178500000 4.2326531763361395
178600000 4.374749104494919
178700000 2.5852246798250147
178800000 4.701312150182972
178900000 4.758366708523873
179000000 4.721187514347795
179100000 4.410051036604513
179200000 3.8183601405350136
179300000 3.521546067353444
179400000 4.20507375186494
179500000 4.175051158448673
179600000 4.480663416993919
179700000 3.8506640426580607
179800000 3.386213231365277
179900000 2.4765

205300000 2.6075664505626595
205400000 2.3305674008911486
205500000 1.5455282550088763
205600000 1.3877738032955098
205700000 2.550776144579275
205800000 2.512059232749929
205900000 2.4566191598539184
206000000 2.476421179087036
206100000 2.321972691223373
206200000 2.470744458530628
206300000 2.535832663357758
206400000 1.512744085733086
206500000 2.1053870540022124
206600000 2.41127944761461
206700000 2.4028373453731935
206800000 2.414669917484664
206900000 2.68497243838494
207000000 2.4264863988410035
207100000 2.4722892592946555
207200000 1.9613923875626127
207300000 1.5880033963191806
207400000 1.3766445501558713
207500000 2.2444504128360516
207600000 2.344314331395636
207700000 2.352414027335207
207800000 2.2690177577650514
207900000 2.134152149896125
208000000 2.146901269143028
208100000 2.1609761289239726
208200000 0.9272729329486191
208300000 2.390720558773359
208400000 2.2917736625668774
208500000 2.160003719868104
208600000 2.2810739084883034
208700000 2.2512298707689844
208

233200000 0.054901015987571744
233300000 0.055910312852786644
233400000 0.11382640359461307
233500000 0.09792138261521857
233600000 0.13880975779360533
233700000 0.11401448377766212
233800000 0.08790708117417163
best so far: 0
type: [1, 1, 1, 121, 1] 121
cases of this type: 1771561
233900000 0.07152591677909757
234000000 0.014705152664889892
234100000 0.017674389229675135
234200000 0.013254803923024071
234300000 0.02157658669599891
234400000 0.020704228355718984
234500000 0.011783518488721716
234600000 0.011344717953066031
234700000 0.016799557793902027
234800000 0.012799586843129662
234900000 0.011040645195454361
235000000 0.010489082629832959
235100000 0.010216990177985694
235200000 0.009873737990051508
235300000 0.00946768018951681
235400000 0.009052371045076184
235500000 0.008477062116622925
235600000 0.01218550833246443
best so far: 0
type: [1, 1, 11, 1, 11] 121
cases of this type: 1771561
235700000 0.053528635260505816
235800000 0.05651030171820521
235900000 0.04497024031994409
2